#   Final Assignment-  Projectial Game (Student Number: 19052055)#

In [1]:
import numpy as np
from vpython import sphere, cylinder, cone, ellipsoid, compound, box, arrow, curve, ring
from vpython import canvas, vector, rate, sleep, label, color, random, cross, mag
#import libries, numpy for calculations, vpython for animations

<IPython.core.display.Javascript object>

### Introduction:
This is the code for a projectile game in which a player aims to topple a wall of randomised distance, we aim to use vpython and projectile motion equations, and some rotational mechanics to simulate this system.

We aim to:
* To create a projectile firing, from a platform; this will be achieved by randomising platform height using `random()` function and the motion will be modelled of kinematic equations of projectile in 2d.
* To detect collision and calculate torque in order to see if the wall topples.

I plan to impliment this with `while` loops for the motion of objects and `if` statments to detect conditions, then by specifying functions of the motion input these to animate the motion.


In [2]:
def objects():
    ''' Function that sets up objects in scene, including ground, wall, platform and labels, also sets up projectile'''
    
    global proj
    global wall
    global ground
    global l_start
    global l_miss
    global l_hit
    global l_ntop
    global l_win
    # variables that are in the function are local, to use them in main code they are made global

    ground= curve(pos= [(0,0,0),(16,0,0)], color=color.green)    
    wall= box(pos= vector(x0,1,0), axis= vector(2,0,0), length= 0.5, height= 2, width= 0.5, mass= 10000, color= color.orange)
    plat= box(pos= vector(0,y0/2,0), axis= vector(y0,0,0), length= 0.3, height= y0, width= 0.3, color= color.white)
    # sets scene with ground, the wall x position,x0, is randomised in main code, along with platform height,y0
    
    l_start=label( pos=vector(0,2,0), text='Enter speed and angle' )
    l_miss=label(pos=vector(8,5,0), text='Miss, failed to hit wall, try again... ')
    l_hit=label( pos=vector(8,5,0), text='You have hit the target')
    l_ntop=label( pos=vector(8,5,0), text='Failed to topple wall, try again... ')
    l_win=label( pos=vector(8,5,0), text='You WIN, Play AGAIN')     
    # set update labels for game 
   
    l_miss.visible=False
    l_hit.visible=False
    l_ntop.visible=False
    l_win.visible=False
    # Clear all labels to be called when appropriate

    if outfit in yes:
        body= sphere(pos= vector(rx0,ry0,0),radius = 0.15, color= color.red)
        eyeR= sphere(pos= vector(rx0+0.07,ry0+0.05,0.07),radius = 0.05, color= color.white)
        eyeL= sphere(pos= vector(rx0+0.07,ry0+0.05,-0.07),radius = 0.05, color= color.white)
        pupilR= sphere(pos = vector(rx0+0.095,ry0+0.06,0.09),radius = 0.02, color= color.black)
        pupilL= sphere(pos = vector(rx0+0.095,ry0+0.06,-0.09),radius = 0.02, color= color.black)
        beak= cone(pos= vector(rx0,ry0,0), axis=vector(0.27,0,0), radius=0.13, color= color.orange)
        comb= ellipsoid(pos= vector(rx0,ry0,0),length=0.3, height=0.45, width=0.1, color= color.orange)
        wingR= box(pos= vector(rx0,ry0,0.1), axis= vector(0,1,-3), length=0.3, height= 0.03, width= 0.11, color= color.red)
        wingL= box(pos= vector(rx0,ry0,-0.1), axis= vector(0,-1,-3), length=0.3, height= 0.03, width= 0.11, color=color.red)
        
        proj= compound([body,eyeR,eyeL,pupilR,pupilL,beak,comb,wingR,wingL], mass=0.1, radius= 0.15, make_trail= True)
        # bird outfit is a compound outfit from contituent objects
      
    elif outfit in no:
        handle= cylinder(pos= vector(0,y0,0), axis= vector(0,0.4,0), radius= 0.05, color= vector(0.72,0.42,0) )
        head= box(pos= vector(0,y0+0.4,0),axis= vector(0.4,0,0),length= 0.3,height= 0.15,width= 0.15,color= color.gray(.6))
        proj= compound([handle, head], mass= 0.1, radius= 0.15, make_trail= True, trail_color= color.red) 
        # hammer is compound of cylinder and box
        
    else:
        proj= sphere(pos= vector(rx0,ry0,0),radius = 0.15, color= color.red, mass=0.1, make_trail=True)
        # regular ball projectile
        
    # if,elif,else- sets the projectile outfit, in main cell input outfit variable is set with output function
    # if/elif statement check if the user input is in the yes/no list which is defined in main cell
      
    proj.trail_color= color.red
    # creates trail for projectile

In [3]:
def initial():
    '''Function to get and initialise projection parameters'''
    
    global v0, theta, px, py, momentum, l_start, confirm
    # variables that are in the function are local, to use them in main code they are made global

    v0= float(input('Input the initial speed as float in metres/second:' ))
    dtheta= float(input('Input the initial angle as float in degrees: '))
    theta= np.radians(dtheta)
    # input for user to initial angle and speed

    px= proj.mass*v0*np.cos(theta)
    py= proj.mass*v0*np.sin(theta)
    momentum= arrow(pos= proj.pos, axis= vector(px,py,0), shaftwidth= 0.035, color= color.red) 
    # calculates x/y momentum and create momentum arrow displaying momentum vector via the axis
    
    scene.waitfor('redraw')
    # forces vpython to wait to redraw canvas, this allows the momentum arrow to display before confirming conditions
    #(to show arrow before proceeding in main code past confirm input), {2}

### Projectile motion:
The path of projectile can be expressed fully by with our initial parameters: the initial speed $v_0$ and the angle of launch $\theta$,.   

The position $r$ of the projectile at any time $t$ is then given by {1}:

$$ r_x = x_0 + v_0t \cos\theta, \quad[1] $$
$$ r_y = y_0 + v_0t \sin\theta −gt^2, \quad[2] $$

From which, as momentum is defined as $ \textbf{p} = m \frac{d \textbf{r}}{dt}, $, we get the following expressions for the momentum of the projectile{1}:

$$ p_x = mv_0 \cos\theta, \quad[3] $$
$$ p_y = mv_0 \sin\theta − mgt, \quad[4] $$

These projectile motion equation are implimented in the function below, this is used in a while loop to calculate the updated position and momentum of the projectile as we iterate of time. The function then updates the vector and projectile's position. 

In addition, for visualisation, I gave the bird a non-real rotation modelling it using angular velocity. From the birds momentum, I calculated translational velocity from which obtained angular velocity $\omega$ from:
$$ v= \frac{\omega}{r}  \quad[5] $$

And as $ \omega =  \frac{d \theta}{dt} $, we get that the change in angle $d\theta$ is :

$$ d\theta= \omega dt \quad[6]$$

The change of angle is then used with the object.rotate function {3}, to rotate the projectile about its centre, however, although a projectile rotates this is due to air resistence and angular momentum give when lauch here this rotation is not physical and for visuals only.

In [4]:
def motion(): 
    '''Function describing equation of motion and momentum of projectile, with input parameters'''
    
    global t, rx, ry, px, py, mag_v
     # variables that are in the function are local, to use them in main code they are made global
        
    t= t+dt
    # when in while loop code iterates over time
    
    rx= rx0 + v0*t*np.cos(theta)
    ry= ry0 + v0*t*np.sin(theta) - 0.5*g*(t**2)
    proj.pos= vector(rx,ry,0)  
    # calculates change in position from eq.[1] & eq.[2], then update projectiles position
    
    px= proj.mass*v0*np.cos(theta)
    py= proj.mass*(v0*np.sin(theta)-g*t)
    momentum.pos= proj.pos
    momentum.axis= vector(px,py,0)
    # calculates change in momentumm from eq.[3] & eq.[4], then update arrows pos. to projectiles pos., and update axis
       
    mag_v= mag(momentum.axis)/proj.mass
    # takes magnitude or arrow axis as balls momentum, from which velocity is obtained
    
    mag_w= mag_v/proj.radius
    b_angle= mag_w*dt
    # calculates angular velocity from eq.[5], and the change in balls angle due to angular velocity from eq.[6]
    
    proj.rotate(angle= -b_angle, axis= vector(0,0,1), origin=vector(rx,0,0))  
    # .rotate function rotates object by specified angle about a axis and origin{3}

### Collision:
The function below allows us to determine the result if there is a collision,we make the following assumptions taken from {1}:
* The collision is completely elastic
* The ball transfers all its momentum to the wall
* The contact time between the ball and the target is finite, and is the same for all collisions. It can thus be represented as a fixed parameter $ \Delta t $ collision.
* The target cannot slide on the ground
* The target has constant density, so that its centre of mass is at the geometric centre of the target

We can use this to calculate the force applied to the target as a result of the collision, and calculate if it is enough to topple the target.

##### Toppling the target:
In order to topple the target over, the applied torque must be sufficient to overcome the restoring torque which operates through the centre of mass,
$$ | \textbf{F}_{applied}| > |\textbf{F}_{restoring}| \quad [7] $$


The magnitude of the restoring torque is given as: 

$$ |\boldsymbol{\tau}_{restoring}| = −mg \frac{w}{2}, \quad[8] $$

where $w$ is the length of the block

And the applied torque supplied by the collision:

$$ \boldsymbol{\tau} _{applied} = \textbf{F}_{applied} × \textbf{d}_a, \quad[9] $$ 

where now $ \boldsymbol{d}_a $ is the vector from the point of rotation to the point of impact. Moreover since we have assumed that all the ball’s momentum is transferred to the target, $\delta p$ will be equal to the momentum of the ball at impact, we get the applied force as:

$$ \textbf{F}_{applied}= \frac{\textbf{p}_{ball} }{\Delta t} \quad[10] $$

Moreover, calculate the angular velocity of the was after the collision, I used the fact the collision is elastic to compare the balls kinetic energy before with the rotational kinetic energy of the wall after, this is given as:

$$\frac{1}{2}m_{ball}v_{ball}^2 = \frac{1}{2}I_{wall}\omega_{wall}^2 \quad[11]$$

Rearranging the equation for the angular velocity after the collision we get:
$$\omega_{wall}= \sqrt{\frac{m_{ball}v_{ball}^2}{I_{wall}}} \quad[12]$$

where $I_{wall}$ is the moment of inertia of the wall, given as:
$$ I_{wall}= \frac{M_{wall}\sqrt{L^2+H^2}}{3}\quad[13]$$

All these variables are calculated by this function and the torques are compared to return a boolean to specify the collision type.


In [5]:
def topple():
    '''Function to calculate impact parameters, and return boolean of succesful or failed colision'''
    
    global mag_W, fall
    # variables that are in the function are local, to use them in main code they are made global
    
    print('\nImpact Height: {0:.2f}m'.format(ry))
    # prints formated impact height
                
    p= momentum.axis
    mag_p= mag(p)
    # momentum given by arrow axis, and absolute value taken
    
    print('The balls momentum is[{0:.1f},{1:.1f},0]Ns'.format(px,py)) 
    print('The magnitude of the momentum is, {0:.2f}Ns'.format(mag_p))
    # prints formated momentum and magnitude of momentum

    Fa= p/deltaT
    da= (proj.pos- vector(x0+0.25,0,0))    
    Ta= cross(Fa,da)
    print('\nApplied Torque from the ball: [{0:.0f},{1:.0f},{2:.0f}]Nm'.format(Ta.x,Ta.y,Ta.z))
    # calculates vector of applied  force and torque, using eq.[9] & eq.[10]

    mag_Ta= mag(Ta)
    print('Magnitude of Applied Torque by the ball: {0:.0f}Nm'.format(mag_Ta))
    # gives absolute value for torque vector

    mag_Tr= wall.mass*g*wall.length/2
    print('Magnitude of Restoring Torque: {0:.0f}Nm'.format(mag_Tr))
    #calculate magnitude of restoring torque from eq.[8]
    
    I= wall.mass*(wall.height**2+wall.length**2)/3   
    mag_W= np.sqrt(proj.mass*(mag_v)**2/I)
    #calculate walls moment of inertia and angular velocity from eq.[12] and eq.[13]

    if mag_Ta > mag_Tr:
        fall= True
    # if contains condition for if wall topples, sets fall true which dictates route code goes in main algorithm, so topples
    
    else:
        fall = False
    # otherwise fall is set false wall will not fall and game resets       

### Wall Toppling
To topple the wall I've modeled the wall as a uniform rode, negating its width, then using a centipedal force due to gravity I have used angular kinematics to calculate the change in angular velocity from which the change in angle is calculated. by resolving gravity of the wall we get the centripeddal force as the component parrel to the walls axis:

$$ F_{cent}= mgsin\theta \rightarrow a_{cent}=gsin\theta \quad [14]$$

here $\theta$ is the angle makes with the horizontal axis as it falls, thus meaning the centripedal acceleration increases to a maximum as it goes from zero to 90 degrees.

To calculate changes in angle  we use the following rotational kinematic equations, analogous to linear suvat equations, $v=u+at$:

$$\omega = \omega + a_{cent}t \quad [15]$$

From this we can find the change in angle using eq.[6] and sum the angles to get $\theta$ over time. This function implements these equations, as we itirate over time, this function also similtaniously drops the ball, using if statement including the projectile motion equations with specific initial conditions of v0=0.

In [6]:
def rotate():
    '''Function to calculate chage in angle and roationof wall'''
    
    global t, w_angle, mag_W, ry, rx,py
    # variables that are in the function are local, to use them in main code they are made global

    t=t+dt
    #iterates over time
  
    acc_W= (g*np.sin(w_angle))
    # acceleration cal. from eq.[14]
    
    mag_W= mag_W + acc_W*(t)
    # new angular velocity from eq. [15]
    
    delta_angle= mag_W*dt
    # change in angle eq.[6]
    
    w_angle= w_angle + delta_angle
    # summing angles as wall topples

    wall.rotate(angle= -delta_angle, axis= vector(0,0,1), origin= vector(x0+0.25,0,0))
    # rotates wall about z axis from right corner 

    if ry >= 0.15:
    # while  ball is above ground the balls position is iterated through freefall eq. of motion
    
        ry= ry - 0.5*g*(t**2)
        proj.pos= vector(rx,ry,0)  
        # calculates change in position from eq.[1] & eq.[2], with v0=0, then update projectiles position
        
        py= -proj.mass*g*t
        momentum.pos= proj.pos
        momentum.axis= vector(0,py,0)
        # calculates change in momentumm from eq.[3] & eq.[4], with v0=0, 
        #then update arrows pos. to projectiles pos., and update axis

### Main Loop:
This code uses a overal `while` loop to allow complete repeats of the game; inside I initalise variables that do not change between attempts then a `while` loop contains the main game code which allows repeats when the player has not won. Once the intial inputs are in, the main games iterates over the main `while` loop when the ball is above the ground. Inside, this loop an `if/else` statement detect if the collision is inside the boundaries of the wall and otherwise it iterates for regular projectile motion using `motion()` function( taking care for under and over shooting). If the ball collides, inside the `if` I use the `topple()` function to calculate collision parameters then comparin torque leads to the final `if/else` statement in which the ball either falls or if torque is greater enough the wall topples.

In [ ]:
repeat= True

while repeat==True:
    x0= 5+ 10*random()
    print('Distance of wall from launch platform:{0:.1f}m'.format(x0))
    y0= random()
    print('Height of launch platform: {0:.2f}m'.format(y0)) 
    # initialise random variables, keeping them constant between attempts

    rx0= 0
    ry0= y0+0.15    
    # initialise balls pos., y.pos has +0.15 to stop it imbedding

    g= 9.81 
    # gravitational acceleration, m/s2
    dt= 0.0001 
    # time interval for loop animation
    deltaT= 0.01
    # contact time of collision
    
    scene= canvas()
    # initialises canvas as while loop deletes canvas initially
    
    yes = {'yes','y', 'ye', 'Y','YES','Yes', 'oui', 'Oui', 'OUI','tak', 'bird', 'Bird','b', 'B'}
    no = {'no','n','N','No', 'NO', 'non','Non', 'NON', 'nie', 'hammer', 'Hammer', 'h', 'H'}
    # List of yes inputs that are accepted, these work yes/no inputs, so they are defined as list of strings
    # Also includes outfit names

    outfit= input('\nDo you want the projectial to be a bird, hammer, or ball:')
    follow= input('Do you want the camera to follow projectial, yes or no ?(By defualt wont follow):')
    # askes out fit and follow inputs, raw input so anything can be put in, but non valid inputs (not y/n) ignored
    
    win= False
    # initialise win as False 

    while win == False:
    # loops until you win
    
        scene.delete()
        #if loops old canvas gets deleted before new one is created with original states
        
        scene= canvas(width= 640, height= 480, center= vector(8,5,0),range= 8)
        # creates the canvas and sets intial variables
        
        objects()
        # calls function to draw objects in the scene
        
        initial()
        # calls function to input initial variables

        confirm= input('Do you want to continue, yes or no, no to change variables:')
        #requests input to continues, if not new variables can be chosen

        l_start.visible= False
        #hide initial label

        ry= 0.15
        rx= 0.15
        t= 0 
        # initialise time and position
        
        miss= True
        #initialies miss, must be changed when hit

        if follow in yes:
            scene.camera.follow(proj)
        # if input yes to follow, centre of canvas is set to proj. position

        if confirm in yes:
        # only allows to continue if confirm input is in yes list
        
            while ry >= 0.15: 
            # repeats until proj hitd ground, y=0.15 is used to stop inbeddment in ground

                if (x0-0.4) <= rx <= (x0+0.25) and ry <= wall.height:
                # hits the wall if y is less than height or if between walls x position
                #(0.4 includes proj radies to stop imbeddmentation in wall)
                    
                    miss= False

                    topple()
                    #calls function to calculate collision parameters
                    
                    t= 0
                    px= 0
                    ry0=ry
                    rx0=rx
                    v0=0
                    theta= np.pi/2
                    w_angle= 0
                    #re initialise conditions of the projectile

                    if fall==True:
                    # fall defined in topple function, if true the topple takes place
                    
                        while wall.pos.y >= 0.25:
                        # as wall falls lower we iterate till it collapses
                            rate(1000)
                            rotate()
                            #calls rotate function to iterate topple over time
                        win= True

                    else:
                        while ry >= 0.15:
                        #iterates till proj hits the ground
                            rate(10000)
                            motion()
                            # calls motion function, with new variables ball falls
                else:
                    rate(1000)
                    motion()
                    # calls projectile motion function to iterate over time

                    if rx > 16 and ry <= 1:
                        ground= curve(pos= [vector(0,0,0),vector(rx+3, 0,0)], color= color.green)
                    # if over shots, extends the ground to new position

            scene.waitfor('redraw') 
            # pauses to redraw before ending/ resetting

            if win == False and miss == False: 
            # outputs code below if hit no topple
            
                l_ntop.visible= True
                # shows label
                
                rx0= 0
                ry0= y0+0.15 
                #r esets initial position
                sleep(1.5)
                #pause before reset
                
            elif win == True and miss == False: 
            # outputs code below if win
                l_win.visible= True
                sleep(1.5) 
                # shows label

                play_again= input('Do you want to play again, yes or no ?:')
                if play_again in no:
                    repeat= False 
                # allows repeat, if input is yes
            else:
            # miss is the only situation left, outputs code below if miss
                l_miss.visible= True
                sleep(1.5) 
                print('Landing distance was {0:.1f}m'.format(rx)) 
                  

Distance of wall from launch platform:7.6m
Height of launch platform: 0.06m


<IPython.core.display.Javascript object>


Do you want the projectial to be a bird, hammer, or ball:ball
Do you want the camera to follow projectial, yes or no ?(By defualt wont follow):yes


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

Input the initial speed as float in metres/second:50
Input the initial angle as float in degrees: 12
Do you want to continue, yes or no, no to change variables:no


<IPython.core.display.Javascript object>

Input the initial speed as float in metres/second:50
Input the initial angle as float in degrees: 15
Do you want to continue, yes or no, no to change variables:yes
Landing distance was 127.6m


<IPython.core.display.Javascript object>

### Conclusion and Dicussion:
The code works well providing the assumptions made, however, lacks a more realistic grounding, it could be improved by:
* Dont consider the ball a point mass and include air resistance, this would give a more physical spin and affect the projectial mostion and range. This could be included by using vectors for the forces on the projectile and then use these to get the corresponding equations of motion for inside the loop.
* Consider both the ball hitting the wall and ground as inelastic collision, including bonce's off of the surfcae's. in addition, this would change the energy transfer  to the wall effecting rotation. For this we would have to include a coefficient of restition of the surfaces.
* Consider restoring torque as a vector, from this we could get expression for resulting torque from the ball and  modelling with shm, could get equation of motion to make the wall wobble at lower torques, and only if the torque goes beyond pivot point there is collapse, this could be done with `while` loop for equation of motion and using and `if` to detect when it is beyond pivot point.
* If we used very large distances we could also take account for the curvature of the earth, this could be done by a more rigid model of gravity, using the gravitational force vector to define the projectiles motion..

# References:
* {1} Dash, L., and Lemos, P., (2019). _"PHAS0007: Final assignment 2019/20"_. UCL Moodle resource available from, https://moodle.ucl.ac.uk/course/view.php?id=10648 \[accessed 17 January 2020\]
* {2}https://www.glowscript.org/docs/GlowScriptDocs/rate.html \[accessed 17 January 2020\]
* {3}https://www.glowscript.org/docs/VPythonDocs/rotation.html \[accessed 17 January 2020\]
* {4}https://www.askiitians.com/forums/Mechanics/fig-shows-a-uniform-block-of-mass-m-and-edge-len_119687.htm \[accessed 17 January 2020\]
* {5}http://astro1.panet.utoledo.edu/~mheben/PHYS_2130/Chapter11-1_mh.pdf \[accessed 17 January 2020\]
* {6}http://www.softschools.com/notes/ap_physics/angular_kinematics/ \[accessed 17 January 2020\]